In [177]:
# import all libraries needed
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from bs4 import Comment
import requests
from webdriver_manager.chrome import ChromeDriverManager
import json


In [140]:
# abbreviations of all NBA teams to iterate through
teams = ['ATL', 'BRK', 'BOS', 'CHO', 'CHI', 
         'CLE', 'DAL', 'DEN', 'DET', 'GSW', 
         'HOU', 'IND', 'LAC', 'LAL', 'MEM', 
         'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 
         'OKC', 'ORL', 'PHI', 'PHO', 'POR', 
         'SAC', 'SAS', 'TOR', 'UTA', 'WAS']

# list to hold info for each baller listed on an nba roster for the 2020 season
all_ballers = []

# list to hold all_ballers without duplicates
ballers = []

# iterate through each team
for team in teams:
    
    # set url for team's basketball reference page in 2020
    url = f'https://www.basketball-reference.com/teams/{team}/2020.html'

    # retrieve page with the requests module
    html = requests.get(url)

    # create BeautifulSoup object; parse with 'html.parser'
    soup = bs(html.text, 'html.parser')

    # scrape the name and bball ref page of each baller listed on the roster
    for x in range(1,50):
        try:
            # append dictionary with baller's scraped data to "ballers"
            all_ballers.append({'Name': soup.find_all('tr')[x].a.text, 
                            'Page': f"https://www.basketball-reference.com{soup.find_all('tr')[x].a['href']}"})
        except:
            # break loop when team's roster has been exhausted
            break

# remove duplicates and append to ballers
for baller in all_ballers:
    if baller not in ballers:
        ballers.append(baller)


In [141]:
len(ballers)

530

In [142]:
# list hold names of ballers who return an error in following loop
nope = []

# iterate through each baller's bballref page
for i in range(0, len(ballers)):

    try:
        print(f"{i}: {ballers[i]['Name']}\n")

        # setup scrape
        url = ballers[i]['Page']
        html = requests.get(url)

        # go to page and scrape
        soup = bs(html.text, 'html.parser')

        # get conventional per game stats and put in pandas df
        stats = pd.read_html(url)[0]

        # filter for full seasons from 2015-2020
        standard = stats.loc[(stats['Season'] == '2019-20') | 
                  (stats['Season'] == '2018-19') |
                  (stats['Season'] == '2017-18') | 
                  (stats['Season'] == '2016-17') | 
                  (stats['Season'] == '2015-16') ]

        # get advanced per game stats and put in pandas df
        table = soup.find_all(string=lambda text: isinstance(text, Comment))[46]
        stats = pd.read_html(table)[0]

        # filter for full seasons from 2015-2020 and remove redundant columns
        advanced = stats.loc[(stats['Season'] == '2019-20') | 
                  (stats['Season'] == '2018-19') |
                  (stats['Season'] == '2017-18') | 
                  (stats['Season'] == '2016-17') | 
                  (stats['Season'] == '2015-16') ].drop(['Age', 'Tm', 'Lg', 'Pos', 'G', 'MP'], axis=1)

        # merge conventional and advanced per game stats into one pandas df
        stats = pd.merge(standard, advanced, on='Season')
        
        # get baller's photo, home city, and home country
        ballers[i]['Photo'] = soup.find_all('img', itemscope="image")[0]['src']
        ballers[i]['City'] = soup.find_all('span', itemprop="birthPlace")[0].text.replace('\n    in\xa0', "").replace('\xa0', " ")
        ballers[i]['State'] = soup.find_all('span', itemprop="birthPlace")[0].a.text
        ballers[i]['Country'] = soup.find_all('span')[13].text
        
        # iterate through each of ballers stats and place in their respective dictionary
        for j in range(0, len(stats.columns)):

            # set variable for statistical category being refered to
            stat = stats.columns[j]

            # most recent biographical/contextual data added without manipulation
            if j < 5:
                ballers[i][stat] = stats[stat][len(stats[stat])-1]

            # games played/started are added as an accumulative sum over last 5 years
            elif j < 7:
                stats[stat] = pd.to_numeric(stats[stat])
                ballers[i][stat] = stats[stat].sum()
                
            # per game stats are added as an average over last 5 years
            else:
                stats[stat] = pd.to_numeric(stats[stat])
                ballers[i][stat] = stats[stat].mean()
        
        # remove unnecessary keys from baller's respective dictionary
        ballers[i].pop('Page')
        ballers[i].pop('Season')
        ballers[i].pop('Unnamed: 24')
        ballers[i].pop('Unnamed: 19')
    
    except:
        # append names of ballers who return an error to "nope"
        nope.append(ballers[i]['Name'])

# show ballers dictionary
ballers


0: DeAndre' Bembry

1: Charlie Brown

2: Vince Carter

3: John Collins

4: Allen Crabbe

5: Dewayne Dedmon

6: Bruno Fernando

7: Brandon Goodwin

8: Treveon Graham

9: Kevin Huerter

10: De'Andre Hunter

11: Damian Jones

12: Alex Len

13: Jabari Parker

14: Chandler Parsons

15: Cam Reddish

16: Jeff Teague

17: Evan Turner

18: Tyrone Wallace

19: Paul Watson

20: Trae Young

21: Jarrett Allen

22: Justin Anderson

23: Wilson Chandler

24: Chris Chiozza

25: Nicolas Claxton

26: Jamal Crawford

27: Spencer Dinwiddie

28: Henry Ellenson

29: Donta Hall

30: Joe Harris

31: Kyrie Irving

32: Tyler Johnson

33: DeAndre Jordan

34: Rodions Kurucs

35: Caris LeVert

36: Timothé Luwawu-Cabarrot

37: Jeremiah Martin

38: Džanan Musa

39: David Nwaba

40: Theo Pinson

41: Taurean Prince

42: Iman Shumpert

43: Garrett Temple

44: Lance Thomas

45: Jaylen Brown

46: Carsen Edwards

47: Tacko Fall

48: Javonte Green

49: Gordon Hayward

50: Enes Kanter

51: Romeo Langford

52: Semi Ojeleye

5

413: Mikal Bridges

414: Jevon Carter

415: Cheick Diallo

416: Jared Harper

417: Ty Jerome

418: Cameron Johnson

419: Frank Kaminsky

420: Jalen Lecque

421: Elie Okobo

422: Kelly Oubre

423: Tariq Owens

424: Cameron Payne

425: Ricky Rubio

426: Dario Šarić

427: Jaylen Adams

428: Carmelo Anthony

429: Trevor Ariza

430: Kent Bazemore

431: Moses Brown

432: Zach Collins

433: Wenyen Gabriel

434: Mario Hezonja

435: Jaylen Hoard

436: Rodney Hood

437: Skal Labissière

438: Damian Lillard

439: Nassir Little

440: CJ McCollum

441: Jusuf Nurkić

442: Anfernee Simons

443: Caleb Swanigan

444: Gary Trent

445: Hassan Whiteside

446: Marvin Bagley

447: Harrison Barnes

448: Nemanja Bjelica

449: Bogdan Bogdanović

450: Corey Brewer

451: Yogi Ferrell

452: De'Aaron Fox

453: Harry Giles

454: Kyle Guy

455: Buddy Hield

456: Richaun Holmes

457: Justin James

458: DaQuan Jeffries

459: Cory Joseph

460: Eric Mika

461: LaMarcus Aldridge

462: Marco Belinelli

463: DeMar DeRozan


[{'Name': "DeAndre' Bembry",
  'Photo': 'https://www.basketball-reference.com/req/202104203/images/players/bembrde01.jpg',
  'City': 'Charlotte, North Carolina',
  'State': 'North Carolina',
  'Country': 'us',
  'Age': 25.0,
  'Tm': 'ATL',
  'Lg': 'NBA',
  'Pos': 'SG',
  'G': 189.0,
  'GS': 23.0,
  'MP': 18.025,
  'FG': 2.25,
  'FGA': 5.075,
  'FG%': 0.44899999999999995,
  '3P': 0.325,
  '3PA': 1.325,
  '3P%': 0.23575,
  '2P': 1.9,
  '2PA': 3.7750000000000004,
  '2P%': 0.5145,
  'eFG%': 0.47950000000000004,
  'FT': 0.65,
  'FTA': 1.125,
  'FT%': 0.53325,
  'ORB': 0.55,
  'DRB': 2.5250000000000004,
  'TRB': 3.075,
  'AST': 1.75,
  'STL': 0.8999999999999999,
  'BLK': 0.375,
  'TOV': 1.3250000000000002,
  'PF': 1.6,
  'PTS': 5.525,
  'PER': 9.2,
  'TS%': 0.49150000000000005,
  '3PAr': 0.246,
  'FTr': 0.21325,
  'ORB%': 3.275,
  'DRB%': 14.95,
  'TRB%': 9.025,
  'AST%': 13.599999999999998,
  'STL%': 2.175,
  'BLK%': 1.75,
  'TOV%': 18.7,
  'USG%': 15.899999999999999,
  'OWS': -0.6,
  'DWS'

In [143]:
# remove nope from ballers
for baller in ballers:
    for no in nope:
        if baller['Name'] == no:
            ballers.remove(baller)
    
            # show baller removed
            print(no)


Sviatoslav Mykhailiuk
Dusty Hannahs


In [196]:
baller_df = pd.DataFrame(ballers)

for stat in baller_df.columns:
    baller_df[stat] = baller_df[stat].fillna(0)

baller_df



,Name,Photo,City,State,Country,Age,Tm,Lg,Pos,G,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,DeAndre' Bembry,https://www.basketball-reference.com/req/20210...,"Charlotte, North Carolina",North Carolina,us,25.0,ATL,NBA,SG,189.0,...,18.700000,15.900000,-0.600000,0.850000,0.250000,0.008000,-3.875000,0.575000,-3.275000,-0.175000
1,Charlie Brown,https://www.basketball-reference.com/req/20210...,"Philadelphia, Pennsylvania",Pennsylvania,us,22.0,ATL,NBA,SG,10.0,...,12.400000,25.100000,0.000000,0.000000,0.000000,-0.028000,-1.100000,-1.400000,-2.500000,0.000000
2,Vince Carter,https://www.basketball-reference.com/req/20210...,"Daytona Beach, Florida",Florida,us,43.0,ATL,NBA,PF,327.0,...,9.160000,16.280000,0.760000,0.980000,1.740000,0.061000,-1.220000,-0.060000,-1.280000,0.360000
3,John Collins,https://www.basketball-reference.com/req/20210...,"Layton, Utah",Utah,us,22.0,ATL,NBA,PF,176.0,...,11.866667,21.433333,4.066667,1.400000,5.433333,0.159000,2.033333,-0.766667,1.266667,1.266667
4,Allen Crabbe,https://www.basketball-reference.com/req/20210...,"Los Angeles, California",California,us,27.0,MIN,NBA,SF,500.0,...,9.246154,13.876923,0.484615,0.423077,0.915385,0.007231,-3.446154,-1.476923,-4.900000,-0.146154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,Admiral Schofield,https://www.basketball-reference.com/req/20210...,"London, United Kingdom",United Kingdom,gb,22.0,WAS,NBA,PF,33.0,...,6.600000,11.900000,0.100000,0.100000,0.100000,0.017000,-4.300000,-1.200000,-5.500000,-0.300000
524,Ish Smith,https://www.basketball-reference.com/req/20210...,"Charlotte, North Carolina",North Carolina,us,31.0,WAS,NBA,PG,749.0,...,13.630769,22.284615,0.369231,1.053846,1.453846,0.047462,-0.230769,-0.800000,-1.061538,0.300000
525,Isaiah Thomas,https://www.basketball-reference.com/req/20210...,"Tacoma, Washington",Washington,us,30.0,WAS,NBA,PG,402.0,...,15.369231,29.023077,1.323077,0.453846,1.753846,0.031615,-0.015385,-3.323077,-3.353846,0.423077
526,Moritz Wagner,https://www.basketball-reference.com/req/20210...,"Berlin, Germany",Germany,de,22.0,WAS,NBA,C,88.0,...,17.200000,20.150000,0.400000,0.550000,0.950000,0.059500,-2.950000,-0.350000,-3.300000,-0.200000


In [254]:
# create dictionaries to hold the ballers' ranks and percentiles in each statistical category
ballers_rank = {}
ballers_percentile = {}

# create keys with lists for each statistical category
for i in range(0, len(baller_df.columns)):
    
    # key for name without manipulation
    if i == 0:
        ballers_rank[baller_df.columns[i]] = []
        ballers_percentile[baller_df.columns[i]] = []
    
    # keys for statistical categories with indication of if they are measuring rank or percentile
    elif (i != 1) | (i != 2) | (i != 3) |(i != 4) | (i != 6) |(i != 7) | (i != 8):
        ballers_rank['Rank_in_' + baller_df.columns[i]] = []
        ballers_percentile['Percentile_in_' + baller_df.columns[i]] = []

# iterate through each statistical category
for i in range(0, len(baller_df.columns)):
    
    # set variable for statistical category being refered to
    stat = baller_df.columns[i]

    #iterate through data for each baller within the current statistical category
    for j in range(0, len(baller_df[stat])):
        
        # append baller's name to corresponding lists
        if i == 0:
            ballers_rank[stat].append(baller_df[stat][j])
            ballers_percentile[stat].append(baller_df[stat][j])

        # determine if current statistical category should hold numeric values
        elif (i != 1) | (i != 2) | (i != 3) |(i != 4) | (i != 6) |(i != 7) | (i != 8):
            
            # sort ballers in ascending order within current statistical category and ensure all values are numbers
            ordered = baller_df.sort_values(stat)[stat].to_numpy()
            
            # iterate through ballers in sorted order
            for k in range(0, len(ordered)):
                
                # determine how many other ballers are below them in current statistical category
                if ordered[k] == baller_df[stat][j]:
                    below = k
                    
                    # break loop once determined
                    break
            
            # calculate rank and percentile
            rank = len(ordered) - below
            percentile = round((below/len(ordered)*100), 2)
            
            # append baller's rank/percentile in current statistical category to corresponding lists
            ballers_rank['Rank_in_' + stat].append(rank)
            ballers_percentile['Percentile_in_' + stat].append(percentile)

# create dataframes out of rank and percentile dictionaries
rank_df = pd.DataFrame(ballers_rank)
percentile_df = pd.DataFrame(ballers_percentile)

# merge all dataframes into one
all_stats = pd.merge(baller_df, rank_df, on='Name')
all_stats = pd.merge(all_stats, percentile_df, on='Name')

# show all_stats
all_stats


,Name,Photo,City,State,Country,Age,Tm,Lg,Pos,G,...,Percentile_in_TOV%,Percentile_in_USG%,Percentile_in_OWS,Percentile_in_DWS,Percentile_in_WS,Percentile_in_WS/48,Percentile_in_OBPM,Percentile_in_DBPM,Percentile_in_BPM,Percentile_in_VORP
0,DeAndre' Bembry,https://www.basketball-reference.com/req/20210...,"Charlotte, North Carolina",North Carolina,us,25.0,ATL,NBA,SG,189.0,...,92.05,31.06,2.65,52.27,23.30,13.83,16.10,74.43,26.52,13.64
1,Charlie Brown,https://www.basketball-reference.com/req/20210...,"Philadelphia, Pennsylvania",Pennsylvania,us,22.0,ATL,NBA,SG,10.0,...,52.27,89.39,19.51,0.19,6.63,6.44,55.30,14.58,34.47,32.20
2,Vince Carter,https://www.basketball-reference.com/req/20210...,"Daytona Beach, Florida",Florida,us,43.0,ATL,NBA,PF,327.0,...,17.05,33.71,55.11,58.71,56.25,38.64,53.41,54.73,54.73,64.39
3,John Collins,https://www.basketball-reference.com/req/20210...,"Layton, Utah",Utah,us,22.0,ATL,NBA,PF,176.0,...,44.70,74.24,94.32,72.35,91.86,89.20,91.29,30.49,85.42,87.31
4,Allen Crabbe,https://www.basketball-reference.com/req/20210...,"Los Angeles, California",California,us,27.0,MIN,NBA,SF,500.0,...,17.61,15.15,46.97,32.39,37.88,13.64,18.56,13.64,13.64,14.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,Admiral Schofield,https://www.basketball-reference.com/req/20210...,"London, United Kingdom",United Kingdom,gb,22.0,WAS,NBA,PF,33.0,...,4.92,6.06,28.41,9.66,14.58,17.99,12.69,18.94,11.17,7.77
524,Ish Smith,https://www.basketball-reference.com/req/20210...,"Charlotte, North Carolina",North Carolina,us,31.0,WAS,NBA,PG,749.0,...,64.20,78.41,41.29,61.74,50.76,29.73,71.97,28.79,58.14,59.85
525,Isaiah Thomas,https://www.basketball-reference.com/req/20210...,"Tacoma, Washington",Washington,us,30.0,WAS,NBA,PG,402.0,...,77.27,96.02,71.21,33.33,56.63,22.54,75.00,2.08,25.95,66.10
526,Moritz Wagner,https://www.basketball-reference.com/req/20210...,"Berlin, Germany",Germany,de,22.0,WAS,NBA,C,88.0,...,86.93,67.05,42.80,37.12,38.64,38.07,27.08,46.21,26.14,10.61


In [255]:
# convert all values for age and games played/started to whole integers
all_stats[['Age', 'G', 'GS']] = all_stats[['Age', 'G', 'GS']].astype(int)

# round all fractional number values to the second decimal place
for i in range(10, 53):
    all_stats[all_stats.columns[i]] = all_stats[all_stats.columns[i]].round(decimals=2)

# show all stats
all_stats


,Name,Photo,City,State,Country,Age,Tm,Lg,Pos,G,...,Percentile_in_TOV%,Percentile_in_USG%,Percentile_in_OWS,Percentile_in_DWS,Percentile_in_WS,Percentile_in_WS/48,Percentile_in_OBPM,Percentile_in_DBPM,Percentile_in_BPM,Percentile_in_VORP
0,DeAndre' Bembry,https://www.basketball-reference.com/req/20210...,"Charlotte, North Carolina",North Carolina,us,25,ATL,NBA,SG,189,...,92.05,31.06,2.65,52.27,23.30,13.83,16.10,74.43,26.52,13.64
1,Charlie Brown,https://www.basketball-reference.com/req/20210...,"Philadelphia, Pennsylvania",Pennsylvania,us,22,ATL,NBA,SG,10,...,52.27,89.39,19.51,0.19,6.63,6.44,55.30,14.58,34.47,32.20
2,Vince Carter,https://www.basketball-reference.com/req/20210...,"Daytona Beach, Florida",Florida,us,43,ATL,NBA,PF,327,...,17.05,33.71,55.11,58.71,56.25,38.64,53.41,54.73,54.73,64.39
3,John Collins,https://www.basketball-reference.com/req/20210...,"Layton, Utah",Utah,us,22,ATL,NBA,PF,176,...,44.70,74.24,94.32,72.35,91.86,89.20,91.29,30.49,85.42,87.31
4,Allen Crabbe,https://www.basketball-reference.com/req/20210...,"Los Angeles, California",California,us,27,MIN,NBA,SF,500,...,17.61,15.15,46.97,32.39,37.88,13.64,18.56,13.64,13.64,14.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,Admiral Schofield,https://www.basketball-reference.com/req/20210...,"London, United Kingdom",United Kingdom,gb,22,WAS,NBA,PF,33,...,4.92,6.06,28.41,9.66,14.58,17.99,12.69,18.94,11.17,7.77
524,Ish Smith,https://www.basketball-reference.com/req/20210...,"Charlotte, North Carolina",North Carolina,us,31,WAS,NBA,PG,749,...,64.20,78.41,41.29,61.74,50.76,29.73,71.97,28.79,58.14,59.85
525,Isaiah Thomas,https://www.basketball-reference.com/req/20210...,"Tacoma, Washington",Washington,us,30,WAS,NBA,PG,402,...,77.27,96.02,71.21,33.33,56.63,22.54,75.00,2.08,25.95,66.10
526,Moritz Wagner,https://www.basketball-reference.com/req/20210...,"Berlin, Germany",Germany,de,22,WAS,NBA,C,88,...,86.93,67.05,42.80,37.12,38.64,38.07,27.08,46.21,26.14,10.61


In [256]:
# filter for all ballers from the US
usa_ballers = all_stats.loc[all_stats['Country'] == 'us', :]

# show all_american_ballers
usa_ballers

,Name,Photo,City,State,Country,Age,Tm,Lg,Pos,G,...,Percentile_in_TOV%,Percentile_in_USG%,Percentile_in_OWS,Percentile_in_DWS,Percentile_in_WS,Percentile_in_WS/48,Percentile_in_OBPM,Percentile_in_DBPM,Percentile_in_BPM,Percentile_in_VORP
0,DeAndre' Bembry,https://www.basketball-reference.com/req/20210...,"Charlotte, North Carolina",North Carolina,us,25,ATL,NBA,SG,189,...,92.05,31.06,2.65,52.27,23.30,13.83,16.10,74.43,26.52,13.64
1,Charlie Brown,https://www.basketball-reference.com/req/20210...,"Philadelphia, Pennsylvania",Pennsylvania,us,22,ATL,NBA,SG,10,...,52.27,89.39,19.51,0.19,6.63,6.44,55.30,14.58,34.47,32.20
2,Vince Carter,https://www.basketball-reference.com/req/20210...,"Daytona Beach, Florida",Florida,us,43,ATL,NBA,PF,327,...,17.05,33.71,55.11,58.71,56.25,38.64,53.41,54.73,54.73,64.39
3,John Collins,https://www.basketball-reference.com/req/20210...,"Layton, Utah",Utah,us,22,ATL,NBA,PF,176,...,44.70,74.24,94.32,72.35,91.86,89.20,91.29,30.49,85.42,87.31
4,Allen Crabbe,https://www.basketball-reference.com/req/20210...,"Los Angeles, California",California,us,27,MIN,NBA,SF,500,...,17.61,15.15,46.97,32.39,37.88,13.64,18.56,13.64,13.64,14.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,Gary Payton,https://www.basketball-reference.com/req/20210...,"Seattle, Washington",Washington,us,27,WAS,NBA,SF,176,...,32.95,41.48,18.94,16.67,17.99,37.31,31.44,76.89,42.23,28.41
522,Justin Robinson,https://www.basketball-reference.com/req/20210...,"Manassas, Virginia",Virginia,us,22,WAS,NBA,PG,9,...,94.70,9.28,19.51,0.19,6.63,13.26,33.33,2.27,10.04,32.20
524,Ish Smith,https://www.basketball-reference.com/req/20210...,"Charlotte, North Carolina",North Carolina,us,31,WAS,NBA,PG,749,...,64.20,78.41,41.29,61.74,50.76,29.73,71.97,28.79,58.14,59.85
525,Isaiah Thomas,https://www.basketball-reference.com/req/20210...,"Tacoma, Washington",Washington,us,30,WAS,NBA,PG,402,...,77.27,96.02,71.21,33.33,56.63,22.54,75.00,2.08,25.95,66.10


In [257]:
# create city_df
city_df = baller_df.loc[baller_df['Country'] == 'us', :].groupby('City').mean()
city_df.reset_index(inplace=True)

# create dictionaries to hold the cities' ranks and percentiles in each statistical category
city_rank = {}
city_percentile = {}

# create keys with lists for each statistical category
for i in range(0, 47):
    
    # key for name without manipulation
    if i == 0:
        city_rank[city_df.columns[i]] = []
        city_percentile[city_df.columns[i]] = []
    
    # keys for statistical categories with indication of if they are measuring rank or percentile
    else:
        city_rank['Rank_in_' + city_df.columns[i]] = []
        city_percentile['Percentile_in_' + city_df.columns[i]] = []

# iterate through each statistical category
for i in range(0, 47):
    
    # set variable for statistical category being refered to
    stat = city_df.columns[i]

    #iterate through data for each city within the current statistical category
    for j in range(0, len(city_df[stat])):
        
        # append city's name to corresponding lists
        if i == 0:
            city_rank[stat].append(city_df[stat][j])
            city_percentile[stat].append(city_df[stat][j])

        # run caluculations for all statistical categoreis
        else:
            
            # sort cities in ascending order within current statistical category and ensure all values are numbers
            ordered = city_df.sort_values(stat)[stat].to_numpy()
            
            # iterate through cities in sorted order
            for k in range(0, len(ordered)):
                
                # determine how many other cities are below them in current statistical category
                if ordered[k] == city_df[stat][j]:
                    below = k
                    
                    # break loop once determined
                    break
            
            # calculate rank and percentile
            rank = len(ordered) - below
            percentile = round((below/len(ordered)*100), 2)
            
            # append city's rank/percentile in current statistical category to corresponding lists
            city_rank['Rank_in_' + stat].append(rank)
            city_percentile['Percentile_in_' + stat].append(percentile)

# create dataframe for number of ballers in each city
baller_counts = pd.DataFrame(usa_ballers['City'].value_counts())
baller_counts.reset_index(inplace=True)
baller_counts = baller_counts.rename(columns={'index':'City','City':'Baller_Counts'})

# create dataframes out of rank and percentile dictionaries
rank_df = pd.DataFrame(city_rank)
percentile_df = pd.DataFrame(city_percentile)

# merge all dataframes into one
city_df = pd.merge(city_df, rank_df, on='City')
city_df = pd.merge(city_df, percentile_df, on='City')
city_df = pd.merge(city_df, baller_counts, on='City')

# show all_stats
city_df



,City,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,...,Percentile_in_USG%,Percentile_in_OWS,Percentile_in_DWS,Percentile_in_WS,Percentile_in_WS/48,Percentile_in_OBPM,Percentile_in_DBPM,Percentile_in_BPM,Percentile_in_VORP,Baller_Counts
0,"Akron, Ohio",31.00,427.333333,289.333333,30.163590,7.359487,14.550256,0.517446,2.023590,5.376923,...,94.89,98.72,94.47,98.72,98.72,98.72,86.38,99.57,99.57,3
1,"Albany, New York",21.00,131.000000,107.000000,29.350000,4.100000,9.850000,0.416000,2.050000,5.350000,...,32.34,55.32,36.60,50.21,22.98,57.45,22.55,42.98,26.81,1
2,"Ames, Iowa",27.00,756.000000,749.000000,33.261538,5.669231,13.038462,0.436615,1.938462,4.907692,...,75.74,81.28,72.77,77.87,49.36,80.00,15.74,57.02,62.98,1
3,"Anaheim, California",22.00,162.000000,149.000000,32.200000,4.066667,10.466667,0.389667,1.866667,5.633333,...,45.96,12.34,91.49,60.43,31.06,68.09,89.79,82.55,85.11,1
4,"Arlington, Massachusetts",27.00,283.000000,12.000000,13.940000,1.600000,3.640000,0.424600,0.660000,1.960000,...,14.89,56.17,67.66,62.13,48.51,49.36,47.66,48.09,60.43,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,"Wilmington, North Carolina",27.00,47.000000,2.000000,13.166667,1.866667,4.333333,0.388667,0.466667,1.366667,...,53.62,27.23,11.91,19.57,25.96,28.51,62.98,30.21,37.87,1
231,"Winston-Salem, North Carolina",25.75,122.750000,85.500000,20.650000,3.290000,7.032500,0.464925,0.870000,2.252500,...,57.02,85.96,68.94,79.57,76.60,68.51,87.66,78.72,85.53,4
232,"Wynnewood, Pennsylvania",32.00,569.000000,285.000000,23.600000,3.323077,8.284615,0.396923,2.415385,6.492308,...,38.72,63.40,54.04,57.45,58.30,76.60,58.30,71.06,64.26,1
233,"York, Maine",25.00,88.000000,69.000000,20.200000,2.800000,6.250000,0.430500,2.200000,5.300000,...,20.85,82.13,54.47,73.19,56.17,62.13,37.87,52.77,70.64,1


In [258]:
# convert all values for age and games played/started to whole integers
city_df[['Age', 'G', 'GS']] = city_df[['Age', 'G', 'GS']].astype(int)

# round all fractional number values to the second decimal place
for i in range(1, 47):
    city_df[city_df.columns[i]] = city_df[city_df.columns[i]].round(decimals=2)

# show city_df
city_df

,City,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,...,Percentile_in_USG%,Percentile_in_OWS,Percentile_in_DWS,Percentile_in_WS,Percentile_in_WS/48,Percentile_in_OBPM,Percentile_in_DBPM,Percentile_in_BPM,Percentile_in_VORP,Baller_Counts
0,"Akron, Ohio",31,427,289,30.16,7.36,14.55,0.52,2.02,5.38,...,94.89,98.72,94.47,98.72,98.72,98.72,86.38,99.57,99.57,3
1,"Albany, New York",21,131,107,29.35,4.10,9.85,0.42,2.05,5.35,...,32.34,55.32,36.60,50.21,22.98,57.45,22.55,42.98,26.81,1
2,"Ames, Iowa",27,756,749,33.26,5.67,13.04,0.44,1.94,4.91,...,75.74,81.28,72.77,77.87,49.36,80.00,15.74,57.02,62.98,1
3,"Anaheim, California",22,162,149,32.20,4.07,10.47,0.39,1.87,5.63,...,45.96,12.34,91.49,60.43,31.06,68.09,89.79,82.55,85.11,1
4,"Arlington, Massachusetts",27,283,12,13.94,1.60,3.64,0.42,0.66,1.96,...,14.89,56.17,67.66,62.13,48.51,49.36,47.66,48.09,60.43,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,"Wilmington, North Carolina",27,47,2,13.17,1.87,4.33,0.39,0.47,1.37,...,53.62,27.23,11.91,19.57,25.96,28.51,62.98,30.21,37.87,1
231,"Winston-Salem, North Carolina",25,122,85,20.65,3.29,7.03,0.46,0.87,2.25,...,57.02,85.96,68.94,79.57,76.60,68.51,87.66,78.72,85.53,4
232,"Wynnewood, Pennsylvania",32,569,285,23.60,3.32,8.28,0.40,2.42,6.49,...,38.72,63.40,54.04,57.45,58.30,76.60,58.30,71.06,64.26,1
233,"York, Maine",25,88,69,20.20,2.80,6.25,0.43,2.20,5.30,...,20.85,82.13,54.47,73.19,56.17,62.13,37.87,52.77,70.64,1


In [259]:
# create state_df
state_df = baller_df.loc[baller_df['Country'] == 'us', :].groupby('State').mean()
state_df.reset_index(inplace=True)

# create dictionaries to hold the states' ranks and percentiles in each statistical category
state_rank = {}
state_percentile = {}

# create keys with lists for each statistical category
for i in range(0, 47):
    
    # key for name without manipulation
    if i == 0:
        state_rank[state_df.columns[i]] = []
        state_percentile[state_df.columns[i]] = []
    
    # keys for statistical categories with indication of if they are measuring rank or percentile
    else:
        state_rank['Rank_in_' + state_df.columns[i]] = []
        state_percentile['Percentile_in_' + state_df.columns[i]] = []

# iterate through each statistical category
for i in range(0, 47):
    
    # set variable for statistical category being refered to
    stat = state_df.columns[i]

    #iterate through data for each state within the current statistical category
    for j in range(0, len(state_df[stat])):
        
        # append state's name to corresponding lists
        if i == 0:
            state_rank[stat].append(state_df[stat][j])
            state_percentile[stat].append(state_df[stat][j])

        # run caluculations for all statistical categoreis
        else:
            
            # sort state in ascending order within current statistical category and ensure all values are numbers
            ordered = state_df.sort_values(stat)[stat].to_numpy()
            
            # iterate through states in sorted order
            for k in range(0, len(ordered)):
                
                # determine how many other states are below them in current statistical category
                if ordered[k] == state_df[stat][j]:
                    below = k
                    
                    # break loop once determined
                    break
            
            # calculate rank and percentile
            rank = len(ordered) - below
            percentile = round((below/len(ordered)*100), 2)
            
            # append state's rank/percentile in current statistical category to corresponding lists
            state_rank['Rank_in_' + stat].append(rank)
            state_percentile['Percentile_in_' + stat].append(percentile)
            
# create dataframe for number of ballers in each state
baller_counts = pd.DataFrame(usa_ballers['State'].value_counts())
baller_counts.reset_index(inplace=True)
baller_counts = baller_counts.rename(columns={'index':'State','State':'Baller_Counts'})

# create dataframes out of rank and percentile dictionaries
rank_df = pd.DataFrame(state_rank)
percentile_df = pd.DataFrame(state_percentile)

# merge all dataframes into one
state_df = pd.merge(state_df, rank_df, on='State')
state_df = pd.merge(state_df, percentile_df, on='State')
state_df = pd.merge(state_df, baller_counts, on='State')

# show all_stats
state_df


,State,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,...,Percentile_in_USG%,Percentile_in_OWS,Percentile_in_DWS,Percentile_in_WS,Percentile_in_WS/48,Percentile_in_OBPM,Percentile_in_DBPM,Percentile_in_BPM,Percentile_in_VORP,Baller_Counts
0,Alabama,28.800000,361.000000,207.200000,18.510769,2.749231,6.024359,0.442238,0.720769,2.099231,...,47.62,40.48,35.71,40.48,54.76,28.57,50.00,40.48,57.14,5
1,Arizona,22.200000,68.800000,17.000000,18.023333,2.633333,6.106667,0.448733,0.590000,1.913333,...,19.05,7.14,30.95,9.52,11.90,4.76,64.29,11.90,4.76,5
2,Arkansas,24.400000,213.000000,89.800000,18.882872,3.397744,7.623487,0.463449,1.001692,2.760974,...,90.48,66.67,28.57,54.76,23.81,66.67,2.38,26.19,66.67,5
3,California,27.068182,368.840909,180.568182,21.674786,3.639396,8.235190,0.439551,1.078185,3.126397,...,76.19,85.71,85.71,88.10,66.67,80.95,66.67,83.33,90.48,44
4,Colorado,25.000000,152.000000,75.000000,19.566667,2.833333,6.033333,0.474000,0.800000,2.033333,...,33.33,88.10,69.05,92.86,95.24,95.24,88.10,97.62,92.86,1
5,Connecticut,23.500000,119.000000,63.500000,18.087500,2.487500,6.562500,0.352375,0.500000,2.075000,...,95.24,0.00,47.62,2.38,0.00,0.00,95.24,2.38,7.14,2
6,Delaware,23.000000,93.000000,24.000000,19.100000,2.700000,6.150000,0.429000,1.050000,3.400000,...,16.67,21.43,97.62,78.57,88.10,40.48,97.62,90.48,88.10,1
7,District of Columbia,26.750000,254.250000,23.750000,16.570417,2.234167,4.839583,0.466313,0.700417,1.927500,...,11.90,52.38,45.24,50.00,57.14,50.00,21.43,30.95,14.29,4
8,Florida,26.800000,251.066667,88.000000,15.222302,2.224365,5.163698,0.410814,0.721810,2.118159,...,59.52,28.57,7.14,16.67,19.05,9.52,4.76,4.76,19.05,15
9,Georgia,25.692308,322.846154,194.307692,23.293681,3.914658,8.495409,0.440059,0.961117,2.753405,...,85.71,80.95,90.48,83.33,35.71,61.90,7.14,33.33,59.52,13


In [260]:
# convert all values for age and games played/started to whole integers
state_df[['Age', 'G', 'GS']] = state_df[['Age', 'G', 'GS']].astype(int)

# round all fractional number values to the second decimal place
for i in range(1, 47):
    state_df[state_df.columns[i]] = state_df[state_df.columns[i]].round(decimals=2)

# show state_df
state_df


,State,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,...,Percentile_in_USG%,Percentile_in_OWS,Percentile_in_DWS,Percentile_in_WS,Percentile_in_WS/48,Percentile_in_OBPM,Percentile_in_DBPM,Percentile_in_BPM,Percentile_in_VORP,Baller_Counts
0,Alabama,28,361,207,18.51,2.75,6.02,0.44,0.72,2.10,...,47.62,40.48,35.71,40.48,54.76,28.57,50.00,40.48,57.14,5
1,Arizona,22,68,17,18.02,2.63,6.11,0.45,0.59,1.91,...,19.05,7.14,30.95,9.52,11.90,4.76,64.29,11.90,4.76,5
2,Arkansas,24,213,89,18.88,3.40,7.62,0.46,1.00,2.76,...,90.48,66.67,28.57,54.76,23.81,66.67,2.38,26.19,66.67,5
3,California,27,368,180,21.67,3.64,8.24,0.44,1.08,3.13,...,76.19,85.71,85.71,88.10,66.67,80.95,66.67,83.33,90.48,44
4,Colorado,25,152,75,19.57,2.83,6.03,0.47,0.80,2.03,...,33.33,88.10,69.05,92.86,95.24,95.24,88.10,97.62,92.86,1
5,Connecticut,23,119,63,18.09,2.49,6.56,0.35,0.50,2.08,...,95.24,0.00,47.62,2.38,0.00,0.00,95.24,2.38,7.14,2
6,Delaware,23,93,24,19.10,2.70,6.15,0.43,1.05,3.40,...,16.67,21.43,97.62,78.57,88.10,40.48,97.62,90.48,88.10,1
7,District of Columbia,26,254,23,16.57,2.23,4.84,0.47,0.70,1.93,...,11.90,52.38,45.24,50.00,57.14,50.00,21.43,30.95,14.29,4
8,Florida,26,251,88,15.22,2.22,5.16,0.41,0.72,2.12,...,59.52,28.57,7.14,16.67,19.05,9.52,4.76,4.76,19.05,15
9,Georgia,25,322,194,23.29,3.91,8.50,0.44,0.96,2.75,...,85.71,80.95,90.48,83.33,35.71,61.90,7.14,33.33,59.52,13


In [261]:
# export all dataframes as csvs
all_stats.to_csv(r'csv/all_stats.csv')
usa_ballers.to_csv(r'csv/usa_ballers.csv')
state_df.to_csv(r'csv/state_df.csv')
city_df.to_csv(r'csv/city_df.csv')

In [262]:
# export all dataframes as json
with open("json/all_stats.json", "w") as outfile: 
    json.dump(all_stats.to_dict(orient='records'), outfile)

with open("json/usa_ballers.json", "w") as outfile: 
    json.dump(usa_ballers.to_dict(orient='records'), outfile)

with open("json/state_df.json", "w") as outfile: 
    json.dump(state_df.to_dict(orient='records'), outfile)
    
    
with open("json/city_df.json", "w") as outfile: 
    json.dump(city_df.to_dict(orient='records'), outfile)



In [57]:
# DONE!

In [16]:
# can see top 20 ballers in a stat of your choosing if you like
stat = 'PTS'
pd.DataFrame(all_stats[[f'Rank in {stat}', 'Name', "City", stat]]).sort_values(stat, ascending=False).head(20)


,Rank in PTS,Name,City,PTS
515,1,James Harden,"Los Angeles, California",31.78
887,2,Damian Lillard,"Oakland, California",26.96
587,3,Anthony Davis,"Chicago, Illinois",26.48
592,4,LeBron James,"Akron, Ohio",26.38
524,5,Russell Westbrook,"Long Beach, California",26.12
417,6,Stephen Curry,"Akron, Ohio",25.98
263,7,Luka Dončić,"Ljubljana, Slovenia",25.00
710,8,Giannis Antetokounmpo,"Athens, Greece",24.78
69,9,Trae Young,"Lubbock, Texas",24.35
94,10,Kyrie Irving,"Melbourne, Australia",24.08


In [59]:
# can see top 20 USA ballers in a stat of your choosing if you like
stat = 'PTS'
pd.DataFrame(usa_ballers[[f'Rank in {stat}', 'Name', "City", stat]]).sort_values(stat, ascending=False).head(20)

,Rank in PTS,Name,City,PTS
515,1,James Harden,"Los Angeles, California",31.78
886,2,Damian Lillard,"Oakland, California",26.96
587,3,Anthony Davis,"Chicago, Illinois",26.48
592,4,LeBron James,"Akron, Ohio",26.38
524,5,Russell Westbrook,"Long Beach, California",26.12
417,6,Stephen Curry,"Akron, Ohio",25.98
69,9,Trae Young,"Lubbock, Texas",24.35
967,11,Bradley Beal,"St. Louis, Missouri",23.84
544,12,Paul George,"Palmdale, California",23.64
918,13,DeMar DeRozan,"Compton, California",23.42


In [164]:
# can see top 20 USA cities in a stat of your choosing if you like
stat = 'PTS'
pd.DataFrame(city_df[[f'Rank in {stat}', "City", stat]]).sort_values(stat, ascending=False).head(20)

,Rank in PTS,City,PTS
118,1,"Lubbock, Texas",24.35
163,2,"Palmdale, California",23.64
41,3,"Compton, California",23.42
51,4,"Edison, New Jersey",23.12
54,5,"Elmsford, New York",22.77
190,6,"Salisbury, North Carolina",22.50
23,7,"Canton, Ohio",21.68
156,8,"Oklahoma City, Oklahoma",21.11
0,9,"Akron, Ohio",20.29
178,10,"Renton, Washington",19.76


In [166]:
# can see top 20 USA states in a stat of your choosing if you like
stat = 'PTS'
pd.DataFrame(state_df[[f'Rank in {stat}', "State", stat]]).sort_values(stat, ascending=False).head(20)

,Rank in PTS,State,PTS
30,1,Ohio,13.12
14,2,Kentucky,12.07
37,3,Utah,11.60
9,4,Georgia,10.52
25,5,New Jersey,10.34
3,6,California,10.22
12,7,Iowa,9.58
39,8,Washington,9.45
28,9,North Carolina,9.44
31,10,Oklahoma,9.31
